# Covid19 data analysis
This notebook uses the data pulled from the NHS England, google and apple APIs.

## Load useful libraries

In [1]:
import covid_data
import data_tools
import plot_tools
import ARIMA_tools

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## Get the data
The data is automatically downloaded by a gitlab action once a week, and deposited into the "data" folder.

In [2]:
df_england_nhse_feed =  pd.read_csv("data/england_nhse_feed.csv")
df_uk_wide_nhse_feed = pd.read_csv("data/uk_wide_nhse_feed.csv")
df_nhsregion_nhse_feed = pd.read_csv("data/nhsregion_nhse_feed.csv")
df_region_nhse_feed = pd.read_csv("data/region_nhse_feed.csv")
df_utla_nhse_feed = pd.read_csv("data/utla_nhse_feed.csv")
df_ltla_nhse_feed = pd.read_csv("data/ltla_nhse_feed.csv")

In [3]:
df_uk_wide_nhse_feed

,date,name,code,covidOccupiedMVBeds,cumAdmissions,cumAdmissionsByAge,cumCasesByPublishDate,cumCasesBySpecimenDate,cumCasesBySpecimenDateRate,cumDeaths28DaysByDeathDate,...,newCasesByPublishDate,newCasesBySpecimenDate,newDeaths28DaysByDeathDate,newDeaths28DaysByPublishDate,newPillarFourTestsByPublishDate,newPillarOneTestsByPublishDate,newPillarThreeTestsByPublishDate,newPillarTwoTestsByPublishDate,newTestsByPublishDate,plannedCapacityByPublishDate
0,2020-01-30,United Kingdom,K02000001,NaN,NaN,NaN,NaN,2.0,NaN,NaN,...,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-31,United Kingdom,K02000001,NaN,NaN,NaN,2.0,2.0,NaN,NaN,...,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-01,United Kingdom,K02000001,NaN,NaN,NaN,2.0,2.0,NaN,NaN,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-02,United Kingdom,K02000001,NaN,NaN,NaN,2.0,2.0,NaN,NaN,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-03,United Kingdom,K02000001,NaN,NaN,NaN,2.0,2.0,NaN,NaN,...,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
559,2021-08-11,United Kingdom,K02000001,871.0,NaN,NaN,6146800.0,6208371.0,9255.0,130833.0,...,29612.0,32383.0,70.0,104.0,16407.0,104011.0,1338.0,724039.0,845795.0,833003.0
560,2021-08-12,United Kingdom,K02000001,882.0,NaN,NaN,6179506.0,6237738.0,9298.8,130904.0,...,33074.0,29367.0,71.0,94.0,13359.0,87448.0,1312.0,713167.0,815286.0,833019.0
561,2021-08-13,United Kingdom,K02000001,NaN,NaN,NaN,6211868.0,NaN,NaN,130935.0,...,32700.0,NaN,31.0,100.0,NaN,NaN,NaN,NaN,NaN,NaN
562,2021-08-14,United Kingdom,K02000001,NaN,NaN,NaN,6241011.0,NaN,NaN,NaN,...,29520.0,NaN,NaN,93.0,NaN,NaN,NaN,NaN,NaN,NaN


## Wrangle the data
So, it would be interesting to be able to predict the a few things about each region of England:
- COVID Hospital Admissions
- Mechanically ventilated beds (as a measure of critical care)
- Positive tests in the community (this will be a lead indicator for the upper two aswell)

Let's try and get these are the following geographical levels:
- England
  - Regions OR NHS Regions
    - Upper Tier Local Authorities OR Trusts

I've put OR above because the civil and NHS geographics aren't easily exchangeable.
